In [1]:
from __future__ import print_function
import pickle
with open('cache/df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)

In [2]:
from collections import Counter
labels = df_test['label']
label_counter = Counter(labels)  # dict<label, #_of_occurrence>
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy

In [3]:
import numpy as np
def get_accuracy(Y, threshold=0, normalize=True):
    """
    Calculates the accuracy of a prediction, decomposed to labels. Numerical predictions are thresholded.
    Type 0: Leaf accuracy, as used in Deng et al, p13.
    Type 1: Prediction IS the full hierarchy
    Type 2: Prediction COVERS the full hierarchy
    Type 3: Prediction IS any partial hierarchy
    Type 4: Prediction COVERS any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    Note that type 2 and 3 are mutually exclusive.
    Args:
        Y: N * D matrix of prediction. Data type may be either numerical or boolean.
        threshold: binarize numerical prediction for accuracy type 1-4.
        normalize: if set to False, returns positive count instead of accuracy.
    Returns:
        21 * 5 matrix, where the last row denotes overall accuracy.
    """
    is_bool = Y.dtype == np.bool
    by_label = np.zeros((20, 5), dtype=np.uint16)
    for i in range(0, len(Y)):
        l = labels[i]
        y = Y[i]
        if is_bool:  # for boolean y, leaf nodes compete in state space
            if y[l]:
                by_label[l, 0] += 1
        else:
            if np.argmax(y[:20]) == l:  # for numerical y, leaf nodes compete explicitly
                by_label[l, 0] += 1
        y = np.nonzero((y / y.max()) > threshold)[0]  # TODO: is normalization necessary?
        if tuple(y) == id_fh[l]:
            by_label[l, 1:] += 1
        elif set(id_fh[l]) in set(y):
            by_label[l, [2, 4]] += 1
        elif tuple(y) in id_hls[l]:
            by_label[l, [3, 4]] += 1
        elif any(set(x) in set(y) for x in id_hls[l]):
            by_label[l, 4] += 1
    overall = np.sum(by_label, axis=0, dtype=np.uint16)
    if normalize:
        by_label = by_label.astype(np.float32)
        for i in range(0, 20):
            by_label[i] /= label_counter[i]
        overall = overall.astype(np.float32) / len(Y)
    return np.vstack((by_label, overall))

In [4]:
np.set_printoptions(precision=4)
id_name = hex_data['id_name']
def accuracy_str(accuracy):
    """
    Makes a pretty version of @get_accuracy result.
    """
    lines = str(accuracy).split('\n')
    lines[0] = lines[0].replace('[[', '[')
    lines[-1] = lines[-1].replace(']]', ']')
    for i in range(0, 20):
        lines[i] = '{} {}'.format(lines[i].replace(' [', '['), id_name[i])
    lines[-1] = '{} {}'.format(lines[-1].replace(' [', '['), 'overall')
    return '\n'.join(lines)

In [5]:
def opt_model_threshold(model_Y):
    """
    Performes an exaustive search for the model and threshold (as used in @get_accuracy) within [-0.5, 0.5], with a step size of 0.1.
    Search criteria is (type_0 * type_1 * type_3) of a prediction's overall accuracy.
    Args:
        model_Y: M * N * D matrix, where M is the number of models.
    Returns:
        opt_model: index of the optimal model in [0, M).
        thresholds[opt_model]: the optimal threshold.
    """
    def opt_threshold(Y, model):  # @model is just for printing progress
        opt_thres, opt_score = 0, 0
        for t in np.linspace(-0.5, 0.5, num=101):
            print('model={0}, threshold={1:.2f}'.format(model, t), end='\r')
            accuracy = get_accuracy(Y, threshold=t)[-1]
            score = accuracy[0] * accuracy[1] * accuracy[3]
            if score > opt_score:
                opt_thres, opt_score = t, score
        return opt_thres, opt_score
    results = list()
    for i in range(0, len(model_Y)):
        results.append(opt_threshold(model_Y[i], i))
    thresholds, scores = zip(*results)
    opt_model = np.argmax(scores)
    return opt_model, thresholds[opt_model]

In [6]:
def opt_model(model_Y):
    accuracies = [get_accuracy(Y)[-1] for Y in model_Y]
    scores = map(lambda x: x[0] * x[1] * x[3], accuracies)
    return np.argmax(scores)

In [7]:
state_space = hex_data['state_space']
def to_crf(Y, leaf_weight=4):
    w = np.ones(Y.shape[1], dtype=np.float32)
    w[:20] = leaf_weight
    def to_crf_step(y):
        scores = map(lambda s: (w * y)[s].sum(), state_space)
        return state_space[np.argmax(scores)]
    return np.array(map(to_crf_step, Y), dtype=np.bool)

In [8]:
# caffe accuracy
iters = np.linspace(5000, 50000, 10, dtype=np.uint16)
iter_Y = np.load('results/test_caffe_4.npy')
# iter_Y = np.tanh(iter_Y)
# opt_iter, opt_thres = opt_model_threshold(iter_Y)
opt_iter, opt_thres = 9, 0.32
print("opt_iter={0}, opt_threshold={1:.2f}".format(iters[opt_iter], opt_thres))
print(accuracy_str(get_accuracy(iter_Y[opt_iter], threshold=opt_thres)))

opt_iter=50000, opt_threshold=0.32
[ 0.2381  0.      0.      0.2381  0.2381] diningtable
[ 0.0909  0.      0.      0.1591  0.1591] chair
[ 0.038   0.      0.      0.1772  0.1772] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.6596  0.234   0.234   0.234   0.234 ] tvmonitor
[ 0.5146  0.1165  0.1165  0.4369  0.4369] train
[ 0.6282  0.2949  0.2949  0.4231  0.4231] bus
[ 0.5268  0.3036  0.3036  0.5268  0.5268] car
[ 0.5962  0.3462  0.3462  0.4615  0.4615] bicycle
[ 0.4444  0.2037  0.2037  0.5     0.5   ] motorbike
[ 0.8134  0.4104  0.4104  0.4104  0.4104] aeroplane
[ 0.481   0.2785  0.2785  0.2911  0.2911] boat
[ 0.      0.      0.      0.3061  0.3061] cow
[ 0.1667  0.      0.      0.4306  0.4306] horse
[ 0.      0.      0.      0.2162  0.2162] sheep
[ 0.325   0.1063  0.1063  0.7188  0.7188] dog
[ 0.5537  0.2147  0.2147  0.8023  0.8023] cat
[ 0.363   0.1778  0.1778  0.5111  0.5111] bird
[ 0.7759  0.2011  0.2011  0.2701  0.270

In [9]:
# caffe + crf accuracy
iters = np.linspace(5000, 50000, 10, dtype=np.uint16)
iter_Y = np.load('results/test_caffe_4.npy')
# iter_Y = np.tanh(iter_Y)
# opt_iter = opt_model(map(to_crf, iter_Y))
opt_iter = 3
print('opt_iter={}'.format(iters[opt_iter]))
print(accuracy_str(get_accuracy(to_crf(iter_Y[opt_iter]))))

opt_iter=20000
[ 0.381   0.381   0.381   0.381   0.381 ] diningtable
[ 0.1818  0.1818  0.1818  0.1818  0.1818] chair
[ 0.0506  0.0506  0.0506  0.0506  0.0506] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.617   0.617   0.617   0.617   0.617 ] tvmonitor
[ 0.6019  0.6019  0.6019  0.6019  0.6019] train
[ 0.6667  0.6667  0.6667  0.6667  0.6667] bus
[ 0.5446  0.5446  0.5446  0.5446  0.5446] car
[ 0.6538  0.6538  0.6538  0.6538  0.6538] bicycle
[ 0.4815  0.4815  0.4815  0.4815  0.4815] motorbike
[ 0.7015  0.7015  0.7015  0.7015  0.7015] aeroplane
[ 0.443   0.443   0.443   0.443   0.443 ] boat
[ 0.      0.      0.      0.      0.    ] cow
[ 0.25    0.25    0.25    0.25    0.25  ] horse
[ 0.      0.      0.      0.      0.    ] sheep
[ 0.3938  0.3938  0.3938  0.3938  0.3938] dog
[ 0.6836  0.6836  0.6836  0.6836  0.6836] cat
[ 0.5185  0.5185  0.5185  0.5185  0.5185] bird
[ 0.5517  0.5517  0.5517  0.5517  0.5517] person
[ 0.4685  

In [10]:
# svm accuracy
kernels = ['linear', 'poly', 'rbf']
kernel_Y = np.load('results/test_svm_4.npy')
# kernel_Y = kernel_Y / (1 - kernel_Y)  # ratio
# kernel_Y = np.log(kernel_Y)  # log ratio
# opt_kernel, opt_thres = opt_model_threshold(kernel_Y)
opt_kernel, opt_thres = 2, 0.27
print("opt_kernel={0}, opt_threshold={1:.2f}".format(kernels[opt_kernel], opt_thres))
print(accuracy_str(get_accuracy(kernel_Y[opt_kernel], threshold=opt_thres)))

opt_kernel=rbf, opt_threshold=0.27
[ 0.5238  0.2381  0.2381  0.5952  0.5952] diningtable
[ 0.3864  0.1136  0.1136  0.3636  0.3636] chair
[ 0.557   0.2911  0.2911  0.4937  0.4937] sofa
[ 0.6129  0.3871  0.3871  0.5484  0.5484] bottle
[ 0.4     0.2     0.2     0.32    0.32  ] pottedplant
[ 0.7234  0.6809  0.6809  0.7447  0.7447] tvmonitor
[ 0.8641  0.6311  0.6311  0.9029  0.9029] train
[ 0.8846  0.7179  0.7179  0.8974  0.8974] bus
[ 0.7054  0.4821  0.4821  0.8482  0.8482] car
[ 0.6731  0.4423  0.4423  0.6538  0.6538] bicycle
[ 0.7778  0.537   0.537   0.8148  0.8148] motorbike
[ 0.8806  0.7388  0.7388  0.8881  0.8881] aeroplane
[ 0.7848  0.6203  0.6203  0.7468  0.7468] boat
[ 0.4286  0.1837  0.1837  0.6735  0.6735] cow
[ 0.5139  0.3194  0.3194  0.6528  0.6528] horse
[ 0.5676  0.3514  0.3514  0.5135  0.5135] sheep
[ 0.7188  0.475   0.475   0.8438  0.8438] dog
[ 0.8079  0.5537  0.5537  0.8814  0.8814] cat
[ 0.837   0.6667  0.6667  0.8667  0.8667] bird
[ 0.8391  0.5345  0.5345  0.6092  0.609

In [11]:
# svm + crf accuracy
kernels = ['linear', 'poly', 'rbf']
kernel_Y = np.load('results/test_svm_4.npy')
# kernel_Y = kernel_Y / (1 - kernel_Y)  # ratio
# kernel_Y = np.log(kernel_Y)  # log ratio
# opt_kernel = opt_model(map(to_crf, kernel_Y))
opt_kernel = 2
print('opt_kernel={}'.format(kernels[opt_kernel]))
print(accuracy_str(get_accuracy(to_crf(kernel_Y[opt_kernel]))))

opt_kernel=rbf
[ 0.619   0.619   0.619   0.619   0.619 ] diningtable
[ 0.4091  0.4091  0.4091  0.4091  0.4091] chair
[ 0.5823  0.5823  0.5823  0.5823  0.5823] sofa
[ 0.5806  0.5806  0.5806  0.5806  0.5806] bottle
[ 0.4     0.4     0.4     0.4     0.4   ] pottedplant
[ 0.7021  0.7021  0.7021  0.7021  0.7021] tvmonitor
[ 0.9029  0.9029  0.9029  0.9029  0.9029] train
[ 0.8846  0.8846  0.8846  0.8846  0.8846] bus
[ 0.7679  0.7679  0.7679  0.7679  0.7679] car
[ 0.7308  0.7308  0.7308  0.7308  0.7308] bicycle
[ 0.7963  0.7963  0.7963  0.7963  0.7963] motorbike
[ 0.8731  0.8731  0.8731  0.8731  0.8731] aeroplane
[ 0.7595  0.7595  0.7595  0.7595  0.7595] boat
[ 0.5306  0.5306  0.5306  0.5306  0.5306] cow
[ 0.5694  0.5694  0.5694  0.5694  0.5694] horse
[ 0.5946  0.5946  0.5946  0.5946  0.5946] sheep
[ 0.75    0.75    0.75    0.75    0.75  ] dog
[ 0.8249  0.8249  0.8249  0.8249  0.8249] cat
[ 0.8889  0.8889  0.8889  0.8889  0.8889] bird
[ 0.7529  0.7529  0.7529  0.7529  0.7529] person
[ 0.75    